In [1]:
import numpy
from math import pi
%matplotlib inline
from laplace_helper import plot_3D, L2_rel_error
from cg_helper import poisson_2d, p_analytical

In [3]:
# parameters
nx = 101
ny = 101
xmax = 0
xmin = 1
ymin = -0.5
ymax = 0.5

l2_target = 1e-10

# spacing
dx = (xmax-xmin)/(nx-1)
dy = (ymax-ymin)/(ny-1)

# mesh
x = numpy.linspace(xmin,xmax,nx)
y = numpy.linspace(ymin,ymax,ny)
X,Y = numpy.meshgrid(x,y)

# source
L = xmax-xmin
b = -2*(pi/L)**2*numpy.sin(pi*X/L)*numpy.cos(pi*Y/L)

# initialization
p_i = numpy.zeros((ny,nx))

# Analytical solution
pan = p_analytical(X,Y,L)

In [6]:
def steepest_descent_2d(p,b,dx,dy,l2_target):
    """Performs steepest descent relaxation, Assumes Dirichlet BCs p=0
    
    Parameters:
    -----------
    p: 2s array of floats
        initial guess
    b: 2D array of floats
        Source term
    dx: float
        Mesh spacing in the x direction
    dy: float
        mesh spacing in the y direction
    l2_target: float
        iteration exit criterion
        
    Return:
    -------
    p: 2d array of float
        Distribution after relaxation
    """
    ny, nx = p.shape
    r = numpy.zeros((ny,nx))
    Ar = numpy.zeros((ny,nx))
    
    l2_norm = 1
    iterations = 0
    l2_conv = []
    
    # iterations
    while l2_norm > l2_target:
        pd = p.copy()
        
        r[1:-1,1:-1] = b[1:-1,1:-1]*dx**2 + 4*pd[1:-1,1:-1] - \
            pd[1:-1,2:] - pd[1:-1,:-2] - pd[2:,1:-1] - pd[:-2,1:-1]
        
        Ar[1:-1,1:-1] = -4*r[1:-1,1:-1] + r[1:-1,2:] + r[1:-1,:-2] +\
            r[2:,1:-1] + r[:-2,1:-1]
        
        rho = numpy.sum(r*r)
        sigma = numpy.sum(r*Ar)
        alpha = rho/sigma
        
        p = pd + alpha*r
        
        # BCs are automatically applied (Dirichlet)
        
        l2_norm = L2_rel_error(pd,p)
        iterations += 1
        l2_conv.append(l2_norm)
    
    print("Number of SD iterations: {0:d}".format(iterations))
    return p, l2_conv

In [7]:
p, l2_conv = steepest_descent_2d(p_i.copy(),b,dx,dy,l2_target)
L2_rel_error(p,pan)

Number of SD iterations: 2


8.2250762211880153e-05

In [9]:
def conjugate_gradient_2d(p,b,dx,dy,l2_target):
    """Performs cg relaxation
        Assuming Dirichlet BCss p=0
    
    Parameters:
    -----------
    p: 2D array of floats
        initial guess
    b: 2d array of floats
        source term
    dx: float
        mesh spacing in x direction
    dy: float
        mesh spaccing in y direction
    l2_target: float
        exit criterion
    
    Returns:
    --------
    p: 2D array of float
        Distribution after relaxation
    """
    
    nx,ny = p.shape
    r = numpy.zeros((ny,nx))
    Ad = numpy.zeros((ny,nx))
    
    l2_norm = 1
    iterations = 0
    l2_conv = []
    
    # step-0 we compute the initial residual and the first search direction
    r[1:-1,1:-1] = b[1:-1,1:-1]*dx**2 + 4*p[1:-1,1:-1] - \
        p[1:-1,2:] - p[1:-1,:-2] - p[2:,1:-1] - p[:-2,1:-1]
    d = r.copy()
    rho = numpy.sum(r*r)
    Ad[1:-1,1:-1] = -4*d[1:-1,1:-1] + d[1:-1,2:] + d[1:-1,:-2] + \
        d[2:,1:-1] + d[:-2,1:-1]
    sigma = numpy.sum(d*Ad)
    
    # iterations
    while l2_norm > l2_target:
        pk = p.copy()
        rk = r.copy()
        dk = d.copy()
        
        alpha = rho/sigma
        
        p = pk + alpha*dk
        r = rk - alpha*Ad
        
        rhop1 = numpy.sum(r*r)
        beta = rhop1/rho
        rho = rhop1
        
        d = r + beta*dk
        Ad[1:-1,1:-1] = -4*d[1:-1,1:-1] + d[1:-1,2:] + d[1:-1,:-2] + \
            d[2:,1:-1] + d[:-2,1:-1]
        sigma = numpy.sum(d*Ad)
        
        l2_norm = L2_rel_error(pk,p)
        iterations += 1
        l2_conv.append(l2_norm)
    
    print("Number of CG iterations: {0:d}:".format(iterations))
    return p,l2_conv

In [10]:
p, l2_conv = conjugate_gradient_2d(p_i.copy(), b, dx, dy, l2_target)
L2_rel_error(p,pan)

Number of CG iterations: 2:


8.2250762211881088e-05

In [11]:
p, l2_conv = poisson_2d(p_i.copy(), b, dx, dy, l2_target)

Number of Jacobi iterations: 31227


In [12]:
b = (numpy.sin(pi*X/L)*numpy.cos(pi*Y/L)+
     numpy.sin(6*pi*X/L)*numpy.sin(6*pi*Y/L))

In [13]:
p, l2_conv = poisson_2d(p_i.copy(), b, dx, dy, l2_target)

p, l2_conv = steepest_descent_2d(p_i.copy(), b, dx, dy, l2_target)

p, l2_conv = conjugate_gradient_2d(p_i.copy(), b, dx, dy, l2_target)

Number of Jacobi iterations: 31226
Number of SD iterations: 29005
Number of CG iterations: 3:
